In [ ]:
import pandas
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pandas.read_excel('output/results.xlsx', sheet_name="data", index_col=0)

# For now, exclude the MindtPy results
df = df.loc[~df.solver.isin(['MindtPy-OA-BM', 'MindtPy-OA-HR'])]

In [ ]:
# Set up line styles
black = "#000000"
gray = "#cccccc"
colors = [
    "#0070a3",
    "#5273ba",
    "#916fc2",
    "#ca67b7",
    "#f6619c",
    "#ff6974",
    "#ff8347",
    "#ffa600",
]

style_map = {
    'Virtual-Best': dict(color=black, linestyle="solid"),
    'Virtual-Worst': dict(color=gray, linestyle="solid"),
    'BARON-BM': dict(color=colors[0], linestyle="dashed"),
    'BARON-HR': dict(color=colors[0], linestyle="dotted"),
    'SCIP-BM': dict(color=colors[1], linestyle="dashed"),
    'SCIP-HR': dict(color=colors[1], linestyle="dotted"),
    'DICOPT-BM': dict(color=colors[2], linestyle="dashed"),
    'DICOPT-HR': dict(color=colors[2], linestyle="dotted"),
    'GDPopt-GLOA': dict(color=colors[3], linestyle="solid"),
    'GDPopt-GLOA-DVB': dict(color=colors[3], linestyle="dashed"),
    'GDPopt-LBB': dict(color=colors[4], linestyle="dashed"),
    'GDPopt-LOA': dict(color=colors[5], linestyle="dotted"),
    'MindtPy-OA-BM': dict(color=colors[6], linestyle="dashed"),
    'MindtPy-OA-HR': dict(color=colors[6], linestyle="dotted"),
}

In [ ]:
def _create_dataframe(key_column):
    # Determine best and worst times
    unique_models = df.model.unique()
    unique_solvers = df.solver.unique()
    solvers_with_virtual = list(unique_solvers) + ['Virtual-Worst', 'Virtual-Best']
    best_times = {model: df.loc[df.model == model][key_column].min() for model in unique_models}
    worst_times = {
        model: df.loc[df.model == model][key_column].max(skipna=False)
        # Check that all solver were able to address this model
        if df.loc[df.model == model]['solver'].nunique() == len(unique_solvers) else np.nan
        for model in unique_models}

    # Eliminate duplicates (take the best time for each model/solver pair)
    df_times = (
        df
        .sort_values(by=key_column)
        .drop_duplicates(subset=['model', 'solver'], keep='first')
    )[['model', 'solver', key_column]]

    # Create records for the virtual best and virtual worst solvers
    df_best_worst_times = pandas.DataFrame.from_records(
        [{
            'model': model,
            'solver': 'Virtual-Best',
            key_column: best_times[model]
        } for model in unique_models] + [{
            'model': model,
            'solver': 'Virtual-Worst',
            key_column: worst_times[model]
        } for model in unique_models]
    )

    df_times = pandas.concat([df_times, df_best_worst_times], sort=False)
    df_times['time'] = df_times[key_column]

    # Drop model column and sort by time_to_soln (note that duplicates were dropped earlier)
    df_solver_times = df_times[['time', 'solver']].sort_values(by='time').dropna()

    for solver in solvers_with_virtual:
        # Create cumulative sum columns corresponding to each solver
        df_solver_times[solver] = (
            df_solver_times
            .apply(lambda row: 1 if row['solver'] == solver else 0, axis=1)
            .cumsum()
        )

    df_solver_times = (
        # Drop the 'solver' column
        df_solver_times[['time',] + solvers_with_virtual]
        # Remove duplicate times. Data columns are cumulative sums, so keep last duplicate row.
        .drop_duplicates(subset='time', keep='last')
        .set_index('time')
    )
    df_solver_times = (
        df_solver_times
        # Sort by cumulative sum (final row of table)
        .sort_values(df_solver_times.last_valid_index(), axis=1, ascending=False)
    )

    # Print out table
    with pandas.option_context(
            'display.max_rows', None, 'display.max_columns', None, 'expand_frame_repr', False
    ), open(f"output/{key_column.replace('_', '-')}.txt", 'w') as resultsfile:
        print(df_solver_times.to_csv(sep='\t'), file=resultsfile)
    
    return df_solver_times

In [ ]:
# Create plot
df_soln = _create_dataframe("time_to_soln")
plt.figure(dpi=300)
for solver_col in df_soln.columns:
    plt.plot(df_soln[solver_col], **style_map[solver_col])
plt.ylim(bottom=0, top=len(best_soln_times))
plt.suptitle('Time to best known solution')
plt.xlabel('Time (seconds)')
plt.ylabel('Instances solved')
plt.legend(bbox_to_anchor=(1.02, 1), loc="upper left", frameon=False)
plt.savefig("output/time-to-soln.png", bbox_inches='tight')

In [ ]:
# # Determine the best time to optimality for each model
# best_opt_times = {model: df.loc[df.model == model].time_to_opt.min() for model in df.model.unique()}

# # Eliminate duplicates (take the best time for each model/solver pair)
# df_time_to_opt = df.sort_values(by='time_to_opt')
# df_time_to_opt = df_time_to_opt.drop_duplicates(subset=['model', 'solver'], keep='first')
# df_time_to_opt = df_time_to_opt[['model', 'solver', 'time_to_opt']]

# # Generate instances solved vs. time plot
# df_time_to_opt = pandas.concat([df_time_to_opt, pandas.DataFrame.from_records(
#     list(dict(model=model, solver='Virtual-Best', time_to_opt=best_opt_times[model]) for model in best_opt_times),
#     index=range(len(best_opt_times))
# )], sort=False)
# df_time_to_opt = df_time_to_opt[['solver', 'time_to_opt']].sort_values(by='time_to_opt')
# for solver in df_time_to_opt.solver.unique():
#     # Create columns corresponding to each solution strategy
#     df_time_to_opt[solver] = df_time_to_opt.apply(lambda row: 1 if row['solver'] == solver else 0, axis=1)
#     df_time_to_opt[solver] = df_time_to_opt[solver].cumsum()
# df_time_to_opt = df_time_to_opt.fillna(value=float('inf'))
# df_time_to_opt = df_time_to_opt.loc[df_time_to_opt['time_to_opt'] <= 1e6]
# df_time_to_opt = df_time_to_opt[['time_to_opt', ] + list(df_time_to_opt.solver.unique())]
# # Remove duplicate times. Data columns are cumulative sums, so keep last duplicate row.
# df_time_to_opt = df_time_to_opt.drop_duplicates(subset='time_to_opt', keep='last').set_index('time_to_opt')
# df_time_to_opt = df_time_to_opt.sort_values(df_time_to_opt.last_valid_index(), axis=1, ascending=False)
# plt.figure(dpi=300)
# for solver_col in df_time_to_opt.columns:
#     plt.plot(df_time_to_opt[solver_col], **style_map[solver_col])
# plt.ylim(bottom=0, top=len(best_soln_times))
# plt.suptitle('Time to global optimality')
# plt.xlabel('Time (seconds)')
# plt.ylabel('Instances solved')
# plt.legend(bbox_to_anchor=(1.02, 1), loc="upper left", frameon=False)
# plt.savefig('output/time-to-opt.png', bbox_inches='tight')
# with pandas.option_context(
#         'display.max_rows', None, 'display.max_columns', None, 'expand_frame_repr', False
# ), open('output/time-to-opt.log', 'w') as resultsfile:
#     print(df_time_to_opt, file=resultsfile)

# Create plot
df_opt = _create_dataframe("time_to_opt")
plt.figure(dpi=300)
for solver_col in df_opt.columns:
    plt.plot(df_opt[solver_col], **style_map[solver_col])
plt.ylim(bottom=0, top=len(best_soln_times))
plt.suptitle('Time to global optimality')
plt.xlabel('Time (seconds)')
plt.ylabel('Instances solved')
plt.legend(bbox_to_anchor=(1.02, 1), loc="upper left", frameon=False)
plt.savefig("output/time-to-opt.png", bbox_inches='tight')

In [ ]:
# # Determine the best time to an ok solution (10% gap) for each model
# best_ok_times = {model: df.loc[df.model == model].time_to_ok_soln.min() for model in df.model.unique()}

# # Eliminate duplicates (take the best time for each model/solver pair)
# df_time_to_ok = df.sort_values(by='time_to_ok_soln')
# df_time_to_ok = df_time_to_ok.drop_duplicates(subset=['model', 'solver'], keep='first')
# df_time_to_ok = df_time_to_ok[['model', 'solver', 'time_to_ok_soln']]

# # Generate instances solved vs. time plot
# df_time_to_ok = pandas.concat([df_time_to_ok, pandas.DataFrame.from_records(
#     list(dict(model=model, solver='Virtual-Best', time_to_ok_soln=best_ok_times[model]) for model in best_ok_times),
#     index=range(len(best_ok_times))
# )], sort=False)
# df_time_to_ok = df_time_to_ok[['solver', 'time_to_ok_soln']].sort_values(by='time_to_ok_soln')
# for solver in df_time_to_ok.solver.unique():
#     # Create columns corresponding to each solution strategy
#     df_time_to_ok[solver] = df_time_to_ok.apply(lambda row: 1 if row['solver'] == solver else 0, axis=1)
#     df_time_to_ok[solver] = df_time_to_ok[solver].cumsum()
# df_time_to_ok = df_time_to_ok.fillna(value=float('inf'))
# df_time_to_ok = df_time_to_ok.loc[df_time_to_ok['time_to_ok_soln'] <= 1e6]
# df_time_to_ok = df_time_to_ok[['time_to_ok_soln', ] + list(df_time_to_ok.solver.unique())]
# # Remove duplicate times. Data columns are cumulative sums, so keep last duplicate row.
# df_time_to_ok = df_time_to_ok.drop_duplicates(subset='time_to_ok_soln', keep='last').set_index('time_to_ok_soln')
# df_time_to_ok = df_time_to_ok.sort_values(df_time_to_ok.last_valid_index(), axis=1, ascending=False)
# plt.figure(dpi=300)
# for solver_col in df_time_to_ok.columns:
#     plt.plot(df_time_to_ok[solver_col], **style_map[solver_col])
# plt.ylim(bottom=0, top=len(best_soln_times))
# plt.suptitle('Time to acceptable solution')
# plt.xlabel('Time (seconds)')
# plt.ylabel('Instances solved')
# plt.legend(bbox_to_anchor=(1.02, 1), loc="upper left", frameon=False)
# plt.savefig('output/time-to-ok-soln.png', bbox_inches='tight')
# with pandas.option_context(
#         'display.max_rows', None, 'display.max_columns', None, 'expand_frame_repr', False
# ), open('output/time-to-ok-soln.log', 'w') as resultsfile:
#     print(df_time_to_ok, file=resultsfile)

# Create plot
df_ok = _create_dataframe("time_to_ok_soln")
plt.figure(dpi=300)
for solver_col in df_ok.columns:
    plt.plot(df_ok[solver_col], **style_map[solver_col])
plt.ylim(bottom=0, top=len(best_soln_times))
plt.suptitle('Time to acceptable solution')
plt.xlabel('Time (seconds)')
plt.ylabel('Instances solved')
plt.legend(bbox_to_anchor=(1.02, 1), loc="upper left", frameon=False)
plt.savefig("output/time-to-ok-soln.png", bbox_inches='tight')

In [ ]:
number_solved = pandas.concat([df_opt.tail(1), df_soln.tail(1), df_ok.tail(1)], axis=0, ignore_index=True, sort=True).fillna(0).astype('int32')
number_solved.set_index(pandas.Index(["1% gap optimal", "1% gap best known", "10% gap best known",]), inplace=True)
number_solved.sort_values(by=["1% gap optimal", "1% gap best known", "10% gap best known"], axis=1, inplace=True, ascending=False)
number_solved = number_solved.transpose()
percent_solved = number_solved.apply(lambda x: round(x / len(best_soln_times) * 100)).astype('int32')

In [ ]:
(number_solved
 .style
 .bar(color='lightgreen', vmin=0, vmax=len(best_soln_times))
 .set_caption("Number of instances solved out of %s" % len(best_soln_times)))

In [ ]:
(percent_solved
 .style
 .bar(color='lightgreen', vmin=0, vmax=100)
 .set_caption("Percent of instances solved out of %s" % len(best_soln_times)))

In [ ]:
# Output the error messages found in the results
fail_messages = (
    df[df.err_msg.notnull()][['model', 'solver', 'err_msg']]
    .sort_values(by=['model', 'solver'])
    .drop_duplicates(subset=['model', 'solver'], keep='first'))
fail_messages.set_index(['model', 'solver'], inplace=True)
fail_messages.style.set_caption("Error messages by model and solver")

In [ ]:
def generate_solution_rank_table(sort_column='time_to_ok_soln'):
    best_solver_df = df.sort_values(by=sort_column).drop_duplicates(subset=['model', 'solver'], keep='first')
    # df.groupby('model').apply(lambda x: print(x[['model', 'solver', 'time_to_ok_soln']]))
    best_solver_df['rank'] = best_solver_df.groupby('model').cumcount()
    best_solver_df['rank'] += 1
    best_solver_df['solver_time'] = best_solver_df.apply(
        lambda row: (row['solver'] + (" %.2f" % row[sort_column]) if pandas.notna(row[sort_column]) else ""), axis=1)
    return (best_solver_df[['model', 'solver', sort_column, 'rank', 'solver_time']]
    #  .groupby('model').head(3)  # Only display top N
     .sort_values(by=['model', sort_column])
     .pivot(index='model', columns='rank', values='solver_time')
     .fillna("")
     .style.set_caption("Best %s" % sort_column.replace("_", " "))
    )

# generate_solution_rank_table(sort_column='time_to_ok_soln')
# generate_solution_rank_table(sort_column='time_to_soln')
generate_solution_rank_table(sort_column='time_to_opt')

In [ ]:
df[df['model'] == '9PPnex']